In [1]:
# #@test {"skip": true}
# !pip install --quiet --upgrade tensorflow_federated_nightly
# !pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle

tff.backends.reference.set_reference_context()

/home/deravindusilva/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:37: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20210101). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations

import math
import pickle
from matplotlib import pyplot as plt

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
[(x.dtype, x.shape) for x in mnist_train]

[(dtype('uint8'), (60000, 28, 28)), (dtype('uint8'), (60000,))]

In [6]:
NUM_EXAMPLES_PER_USER = 1000
BATCH_SIZE = 100
NUM_OF_CLIENTS = 90
# NUM_OF_CLIENTS_ARRAY = [0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3, 4,4,4,4, 5,5,5,5, 6,6,6,6, 7,7,7,7, 8,8,8,8, 9,9,9,9]
# NUM_OF_CLIENTS_ARRAY = [0,1,2,3,4,5,6,7,8,9]

In [7]:
source_train_X, source_train_Y = shuffle(mnist_train[0], mnist_train[1], random_state=0)
source_test_X, source_test_Y = shuffle(mnist_test[0], mnist_test[1], random_state=0)

In [8]:
def get_data_for_digit_iid(source_X, source_Y):
    output_sequence = []
    for i in range(0, min(NUM_EXAMPLES_PER_USER, len(source_X)), BATCH_SIZE):
        batch_samples_X = source_X[i:i + BATCH_SIZE]
        batch_samples_Y = source_Y[i:i + BATCH_SIZE]
        output_sequence.append({
            'x': np.array([xx.flatten() / 255.0 for xx in batch_samples_X],dtype=np.float32),
            'y': np.array([yy for yy in batch_samples_Y], dtype=np.int32)
        })
    return output_sequence

xtr = int(len(source_train_X)/NUM_OF_CLIENTS)
ytr = int(len(source_train_Y)/NUM_OF_CLIENTS)

xte = int(len(source_test_X)/NUM_OF_CLIENTS)
yte = int(len(source_test_Y)/NUM_OF_CLIENTS)

federated_train_data = [get_data_for_digit_iid(source_train_X[(d*xtr):(d*xtr+xtr)], source_train_Y[(d*ytr):(d*ytr+ytr)]) for d in range(NUM_OF_CLIENTS)]

federated_test_data = [get_data_for_digit_iid(source_test_X[(d*xte):(d*xte+xte)], source_test_Y[(d*yte):(d*yte+yte)]) for d in range(NUM_OF_CLIENTS)]

In [9]:
def get_data_for_digit_niid(source, digit):
    output_sequence = []
    all_samples = [i for i, d in enumerate(source[1]) if d == digit]
    for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
        batch_samples = all_samples[i:i + BATCH_SIZE]
        output_sequence.append({
            'x': np.array([source[0][i].flatten() / 255.0 for i in batch_samples],dtype=np.float32),
            'y': np.array([source[1][i] for i in batch_samples], dtype=np.int32)
        })
    return output_sequence


# federated_train_data = [get_data_for_digit_niid(mnist_train, d) for d in NUM_OF_CLIENTS_ARRAY]

# federated_test_data = [get_data_for_digit_iid(source_test_X[(d*xte):(d*xte+xte)], source_test_Y[(d*yte):(d*yte+yte)]) for d in range(NUM_OF_CLIENTS)]

In [10]:
# mnist_train[0].shape

In [11]:
# federated_train_data

In [12]:
# federated_train_data[0][0].get('y').shape

In [13]:
# source_train_X.shape

In [14]:
# federated_train_data2[0][0].get('x').shape

In [15]:
# federated_train_data2[0][0].get('y').shape

In [16]:
BATCH_SPEC = collections.OrderedDict(
    x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)

str(BATCH_TYPE)

'<x=float32[?,784],y=int32[?]>'

In [17]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

str(LOCAL_DATA_TYPE)

'<x=float32[?,784],y=int32[?]>*'

In [18]:
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[784, 10], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[10], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

<weights=float32[784,10],bias=float32[10]>


In [19]:
SERVER_MODEL_TYPE = tff.FederatedType(MODEL_TYPE, tff.SERVER)
CLIENT_DATA_TYPE = tff.FederatedType(LOCAL_DATA_TYPE, tff.CLIENTS)

Client

In [20]:
@tf.function
def forward_pass(model, batch):
    predicted_y = tf.nn.softmax(tf.matmul(batch['x'], model['weights']) + model['bias'])
    return -tf.reduce_mean(
        tf.reduce_sum(
            tf.one_hot(batch['y'], 10) * tf.math.log(predicted_y), axis=[1]))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
    return forward_pass(model, batch)

In [21]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    
    @tf.function
    def _train_on_batch(model_vars, batch):
        # Perform one step of gradient descent using loss from `batch_loss`.
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars
  
    return _train_on_batch(model_vars, batch)

In [22]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):
    
    # Mapping function to apply to each batch.
    @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
    def batch_fn(model, batch):
        return batch_train(model, batch, learning_rate)
    return tff.sequence_reduce(all_batches, initial_model, batch_fn)

In [23]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
    return tff.sequence_sum(
        tff.sequence_map(
            tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE),all_batches))

Server

In [24]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
    return tff.federated_mean(tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))

In [25]:
SERVER_FLOAT_TYPE = tff.FederatedType(tf.float32, tff.SERVER)


@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE,
                           CLIENT_DATA_TYPE)
def federated_train(model, learning_rate, data):
#     client_output = tff.federated_map(local_train, [
#             tff.federated_broadcast(model),
#              tff.federated_broadcast(learning_rate), data
#         ])
    return tff.federated_mean(tff.federated_map(local_train, [
            tff.federated_broadcast(model),
             tff.federated_broadcast(learning_rate), data
        ]))

In [26]:
initial_model = collections.OrderedDict(
    weights=np.zeros([784, 10], dtype=np.float32),
    bias=np.zeros([10], dtype=np.float32))

In [27]:
def modelR(weights, biases, data):
  count = 0
  avg = 0
  for j in range(len(federated_test_data[0])):
    l = [np.where(i==max(i))[0][0] for i in activations.sigmoid(np.matmul(federated_test_data[0][j].get('x'), model.get('weights'))).numpy()]
    Y = federated_test_data[0][j].get('y')
    
    for i in range(len(Y)):
      avg+=1
      if l[i] == Y[i]:
        count+= 1
  return count/avg

In [28]:
# model = initial_model
# learning_rate = 0.1
# for round_num in range(5):
#     model = federated_train(model, learning_rate, federated_train_data)
#     learning_rate = learning_rate * 0.9
#     loss = federated_eval(model, federated_train_data)
#     print('round {}, loss={}, accuracy={}'.format(round_num, loss, modelR(model.get('weights'), model.get('bias'), federated_test_data)))

In [29]:
accuracy = 0
model = initial_model
learning_rate = 0.1
count = 0

accuracy_list = []
for i in range(10):
    model = federated_train(model, learning_rate, federated_train_data)
    learning_rate = learning_rate * 0.9
    loss = federated_eval(model, federated_train_data)
    accuracy = modelR(model.get('weights'), model.get('bias'), federated_test_data)
    count+=1
    accuracy_list.append(accuracy)
    print('round {}, loss={}, accuracy={}'.format(count, loss, accuracy)) 

pickle.dump(accuracy_list, open('90_clients', 'wb'))

round 1, loss=12.019745826721191, accuracy=0.7837837837837838
round 2, loss=9.80299186706543, accuracy=0.8108108108108109
round 3, loss=8.504426002502441, accuracy=0.8468468468468469
round 4, loss=7.675743103027344, accuracy=0.8558558558558559
round 5, loss=7.1082444190979, accuracy=0.8558558558558559
round 6, loss=6.698227405548096, accuracy=0.8558558558558559
round 7, loss=6.38978385925293, accuracy=0.8648648648648649
round 8, loss=6.150468826293945, accuracy=0.8648648648648649
round 9, loss=5.960265636444092, accuracy=0.8648648648648649
round 10, loss=5.806177616119385, accuracy=0.8558558558558559


In [30]:
plt.figure(figsize=(8,6))
plt.plot(pickle.load(open('20_clients', 'rb')), label="20")
plt.plot(pickle.load(open('30_clients', 'rb')), label="30")
plt.plot(pickle.load(open('50_clients', 'rb')), label="50")
plt.plot(pickle.load(open('90_clients', 'rb')), label="90")


plt.legend(loc="upper left")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '20_clients'

<Figure size 576x432 with 0 Axes>

In [ ]:
# def modelR(weights, biases, data):
#   count = 0
#   avg = 0
#   for j in range(len(federated_test_data[0])):
#     l = [np.where(i==max(i))[0][0] for i in activations.sigmoid(np.matmul(federated_test_data[0][j].get('x'), model.get('weights'))).numpy()]
#     Y = federated_test_data[0][j].get('y')
#     print('l={}, Y={}'.format(l, Y))
    
#     for i in range(len(Y)):
#       avg+=1
#       if l[i] == Y[i]:
#         count+= 1
#   return count/avg


In [ ]:
# modelR(model.get('weights'), model.get('bias'), federated_test_data)

In [33]:
federated_test_data[0]

{'x': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'y': array([8, 7, 1, 7, 1, 2, 1, 9, 5, 4, 5, 6, 3, 2, 0, 7, 6, 4, 6, 4, 5, 7,
        8, 9, 8, 7, 4, 3, 9, 4, 2, 2, 9, 7, 6, 7, 1, 2, 8, 3, 0, 6, 4, 0,
        8, 7, 8, 2, 9, 0, 4, 3, 4, 0, 7, 0, 7, 4, 6, 1, 9, 7, 2, 8, 8, 0,
        1, 0, 5, 1, 5, 1, 4, 9, 4, 8, 7, 3, 7, 8, 9, 1, 4, 4, 0, 2, 5, 7,
        7, 5, 3, 0, 6, 2, 4, 1, 6, 8, 6, 9], dtype=int32)}